In [68]:
from os import chdir, path, getcwd
for i in range(10):
    if path.isfile("checkcwd"):
        break
    chdir(path.pardir)
if path.isfile("checkcwd"):
    pass
else:
    raise Exception("Something went wrong. cwd=" + getcwd())

In [69]:
import pandas as pd
data: pd.DataFrame = pd.read_pickle('assets/cleaned_data.pkl')

In [70]:
data.columns

Index(['year', 'is_extended', 'city', 'country_txt', 'region_txt', 'summary',
       'success', 'attack_tactic', 'attack_type', 'suicide', 'target_type',
       'target_subcat', 'te_group', 'te_motive', 'weapon_type',
       'weapon_subtype', 'total_killed', 'total_wounded', 'te_killed',
       'te_wounded', 'prorperty_damage', 'ishostkid', 'ransom', 'ransom_amt',
       'ransom_amt_paid', 'hostage_outcome', 'civ_killed', 'civ_wounded',
       'total_casualties', 'country_code'],
      dtype='object')

In [71]:
import numpy as np

kill_data = data[['year', 'country_code', 'country_txt', 'total_casualties']]
kill_data = kill_data.groupby(['year', 'country_code', 'country_txt'])['total_casualties'].sum().reset_index()
kill_data = kill_data.rename(columns={'country_txt': 'Country'})

all_countries = kill_data['country_code'].unique()
all_years = all_years = np.arange(1970, 2021)


all_combinations = pd.MultiIndex.from_product([all_years, all_countries], names=['year', 'country_code']).to_frame(index=False)
complete_data = pd.merge(all_combinations, kill_data, on=['year', 'country_code'], how='left')

complete_data['total_casualties'] = complete_data['total_casualties'].fillna(0)

country_names = kill_data[['country_code', 'Country']].drop_duplicates()
complete_data = pd.merge(complete_data, country_names, on='country_code', how='left')

complete_data = complete_data.sort_values(['year', 'country_code']).reset_index(drop=True)
complete_data.drop_duplicates(inplace=True)
complete_data.drop(['Country_x'], axis=1,  inplace=True)
complete_data.rename(columns={'Country_y': 'Country'}, inplace=True)

log_bin_edges = [0, 3, 10, 30, 100, 300, 1000, 3000, 10000, 30000000]
log_bin_labels = ['0-3', '3-10', '10-30', '30-100', '100-300', '300-1000', '1000-3000', '3000-10000', '>10000']
complete_data['total_casualties_cat'] = pd.cut(complete_data['total_casualties'], bins=log_bin_edges, labels=log_bin_labels, include_lowest=True)

complete_data.sort_values(['year', 'total_casualties'], inplace=True)
complete_data.reset_index(drop=True, inplace=True)
complete_data

,year,country_code,total_casualties,Country,total_casualties_cat
0,1970,AFG,0.0,Afghanistan,0-3
1,1970,AGO,0.0,Angola,0-3
2,1970,ALB,0.0,Albania,0-3
3,1970,AND,0.0,Andorra,0-3
4,1970,ARE,0.0,United Arab Emirates,0-3
...,...,...,...,...,...
9787,2020,IRQ,1617.0,Iraq,1000-3000
9788,2020,SYR,1701.0,Syria,1000-3000
9789,2020,YEM,1793.0,Yemen,1000-3000
9790,2020,NGA,2903.0,Nigeria,1000-3000


In [72]:
import plotly.express as px
import numpy as np

fig = px.choropleth(complete_data,
                    locations='country_code',
                    color='total_casualties',
                    hover_data={'Country': True, 'total_casualties': True, 'country_code': False, 'year': False, 'total_casualties_cat': False},
                    range_color=(0, np.max(complete_data['total_casualties'])),
                    animation_frame='year',
                    color_continuous_scale='hot',
                    color_discrete_sequence=['#0d0887', '#46039f', '#7201a8', '#9c179e', '#bd3786', '#d8576b', '#ed7953', '#fb9f3a', '#fdca26', '#f0f921'],
                    labels={'total_casualties':'Total number of casualties'},
                    template='plotly_dark'
)

sliders=[{
    "active": 0,
    "currentvalue": {"prefix": "Year "},
    "steps": fig.layout.sliders[0].steps,
}]


fig.update_layout(
    plot_bgcolor='black',
    margin={"r":0,"t":10,"l":0,"b":0},
    sliders=sliders,
)


# fig.update_geos(
#     showcoastlines=True, coastlinecolor="white",
#     showland=True, landcolor="gray",
#     showocean=True, oceancolor="gray",
#     showlakes=True, lakecolor="gray",
#     showrivers=True, rivercolor="gray"
# )
fig.update_layout(
    coloraxis_colorbar=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.89
    )
)


fig.update_traces(marker_line_width=0)

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 120
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 30

# to remove play button
# fig["layout"].pop("updatemenus")

fig.show()



In [135]:
all_kill_data = data[['year', 'country_code', 'country_txt', 'total_casualties']]
all_kill_data = all_kill_data.groupby(['country_code', 'country_txt'])['total_casualties'].sum().reset_index()

log_bin_edges2 = [0, 30, 100, 300, 1000, 3000, 10000, 30000, 100000, 300000000]
log_bin_labels2 = ['0-30', '30-100', '100-300', '300-1000', '1000-3000', '3000-10000', '10000-30000', '30000-100000', '>100000']
all_kill_data['total_casualties_cat'] = pd.cut(all_kill_data['total_casualties'], bins=log_bin_edges2, labels=log_bin_labels2, include_lowest=True)

all_kill_data.sort_values(['total_casualties'], inplace=True)
all_kill_data.reset_index(drop=True, inplace=True)
all_kill_data = all_kill_data.rename(columns={'country_txt': 'Country'})

In [136]:
fig = px.choropleth(all_kill_data[['country_code', 'total_casualties_cat', 'Country']],
                    locations='country_code',
                    hover_data={'Country': True, 'country_code': False, 'total_casualties_cat': True},
                    color='total_casualties_cat',
                    color_discrete_sequence=['#0d0887', '#46039f', '#7201a8', '#9c179e', '#bd3786', '#d8576b', '#ed7953', '#fb9f3a', '#fdca26', '#f0f921'],
                    labels={'total_casualties_cat':'Total number of casualties'},
                    template='plotly_dark',
)

# sliders=[{
#     "active": 0,
#     "currentvalue": {"prefix": "Year "},
#     "steps": fig.layout.sliders[0].steps,
# }]


fig.update_layout(
    plot_bgcolor='black',
    margin={"r":0,"t":0,"l":0,"b":0},
    # sliders=sliders,
)

fig.update_traces(marker_line_width=0)

# fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 120
# fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 30

# to remove play button
# fig["layout"].pop("updatemenus")

fig.show()


In [104]:
pd.options.mode.copy_on_write = True

In [130]:
data_num_atk = data[['region_txt', 'country_txt', 'country_code']]
data_num_atk.loc[data_num_atk['country_txt'] == 'Germany', 'region_txt'] = 'Western Europe' # damn
data_num_atk.loc[data_num_atk['region_txt'] == 'Middle East & North Africa', 'region_txt'] = 'Middle East<br>& North Africa'
data_num_atk.loc[data_num_atk['region_txt'] == 'Central America & Caribbean', 'region_txt'] = 'Central America<br>& Caribbean'
data_num_atk = data_num_atk.groupby(['country_txt', 'region_txt']).size().reset_index(name='Number of attacks')

fig3 = px.sunburst(data_num_atk,
                   path=['region_txt', 'country_txt'], values='Number of attacks',
                   color='Number of attacks',
                   hover_data={'Number of attacks': True},
                   color_continuous_scale='RdBu_r',
                   title='Number of attacks per region',
                   template='plotly_dark'
                  )

fig3.data[0].hovertemplate = '%{label}<br>Number of attacks = %{value}<extra></extra>'

fig3.show()